In [4]:
import pandas as pd
import random
from collections import defaultdict, namedtuple

In [5]:
# =========================================================================
# PARÂMETROS GERAIS
# =========================================================================
dias_semana = ["SEG", "TER", "QUA", "QUI", "SEX"]
tempos = [1,2,3,4,5]

In [6]:
# =========================================================================
# 1) LEITURA DOS ARQUIVOS
# =========================================================================
df_prof = pd.read_excel("professores.xlsx")
df_demanda_raw = pd.read_excel("demanda.xlsx")

In [7]:
# =========================================================================
# 2) CRIAR DICIONÁRIO DE DISPONIBILIDADE
#    disponib[prof][turno][dia][tempo] = True/False
# =========================================================================
disponib = {}
for i, row in df_prof.iterrows():
    servidor = row["SERVIDOR"]
    turno    = row["TURNO"]
    if servidor not in disponib:
        disponib[servidor] = {}
    if turno not in disponib[servidor]:
        disponib[servidor][turno] = {}
    for d in dias_semana:
        if d not in disponib[servidor][turno]:
            disponib[servidor][turno][d] = {}
        for t in tempos:
            col = f"{d}{t}"
            if col in row:
                val = row[col]
                disponib[servidor][turno][d][t] = (val == 1)
            else:
                disponib[servidor][turno][d][t] = False

In [8]:
# =========================================================================
# 3) FILTRAR "APOIO" E MONTAR DEMANDA
# =========================================================================
df_demand = df_demanda_raw[df_demanda_raw["DISC"] != "APOIO"].copy()
Pedido = namedtuple("Pedido", "prof turno nivel serie turma disc ch")
demanda = []
for i, row in df_demand.iterrows():
    demanda.append(Pedido(
        prof  = row["SERVIDOR"],
        turno = row["TURNO"],
        nivel = row["NIVEL"],
        serie = str(row["SERIE/ANO"]),
        turma = str(row["TURMA"]),
        disc  = row["DISC"],
        ch    = int(row["CH.TURMA"])
    ))

In [9]:
# =========================================================================
# 4) ESTRUTURA DE ALOCAÇÃO
#    A estrutura:
#      horario[turno][nivel][dia][(serie,turma)] = { tempo: (prof,disc) }
# =========================================================================
def cria_horario_vazio(demanda):
    """Gera a estrutura vazia para todos os (turno, nivel, dia, serie, turma) da demanda."""
    horario = {}
    combos = set()
    for ped in demanda:
        combos.add((ped.turno, ped.nivel))
    for (tnr, niv) in combos:
        if tnr not in horario:
            horario[tnr] = {}
        if niv not in horario[tnr]:
            horario[tnr][niv] = {}
        for d in dias_semana:
            horario[tnr][niv][d] = {}
    for ped in demanda:
        for d in dias_semana:
            if (ped.serie, ped.turma) not in horario[ped.turno][ped.nivel][d]:
                horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)] = {}
    return horario

In [10]:
# =========================================================================
# 5) FUNÇÕES DE VERIFICAÇÃO DE REGRAS (heurísticas)
# =========================================================================
def professor_disponivel(prof, turno, dia, tempo):
    if prof not in disponib: return False
    if turno not in disponib[prof]: return False
    return disponib[prof][turno][dia][tempo]

def professor_ocupado_esse_tempo(horario, prof, turno, nivel, dia, tempo):
    for (st, dicT) in horario[turno][nivel][dia].items():
        if tempo in dicT:
            (p, _) = dicT[tempo]
            if p == prof:
                return True
    return False

def turma_ja_tem_aula(horario, turno, nivel, dia, serie, turma, tempo):
    return (tempo in horario[turno][nivel][dia][(serie, turma)])

def check_buraco(horario, ped, dia):
    dicT = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if not dicT: return False
    tempos_ocup = sorted(dicT.keys())
    if tempos_ocup[0] != 1: return True
    maxi = max(tempos_ocup)
    for x in range(1, maxi+1):
        if x not in dicT:
            return True
    return False

def professor_turma_ja_usou_dia(horario, ped, dia):
    """Regra: para o mesmo pedido, o professor não pode ter mais de 1 tempo no mesmo dia."""
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    for (tm, (p, _)) in dicTurma.items():
        if p == ped.prof:
            return True
    return False

def alocar_1tempo(horario, ped, dia, tempo):
    horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)][tempo] = (ped.prof, ped.disc)

def desalocar_1tempo(horario, ped, dia, tempo):
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if tempo in dicTurma:
        del dicTurma[tempo]

def pode_alocar(horario, ped, dia, tempo):
    # Verifica disponibilidade e se professor não está em outra turma no mesmo slot
    if not professor_disponivel(ped.prof, ped.turno, dia, tempo):
        return False
    if professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, dia, tempo):
        return False
    # Regra: cada professor não pode ter mais de 1 tempo na mesma turma no mesmo dia
    if professor_turma_ja_usou_dia(horario, ped, dia):
        return False
    # Testa alocação temporária para verificar buraco
    alocar_1tempo(horario, ped, dia, tempo)
    if check_buraco(horario, ped, dia):
        desalocar_1tempo(horario, ped, dia, tempo)
        return False
    return True

In [15]:
# =========================================================================
# 6) HEURÍSTICA PRINCIPAL DE ALOCAÇÃO (iterada)
# =========================================================================
NUM_ITERACOES = 10000  # Número de iterações da busca heurística

def alocar_demanda(horario, demanda):
    nao_alocados = []
    dem_local = list(demanda)
    random.shuffle(dem_local)
    for ped in dem_local:
        ch_rest = ped.ch
        dias_rand = list(dias_semana)
        random.shuffle(dias_rand)
        for d in dias_rand:
            if ch_rest <= 0:
                break
            tempos_rand = list(tempos)
            random.shuffle(tempos_rand)
            for t in tempos_rand:
                if ch_rest <= 0:
                    break
                if pode_alocar(horario, ped, d, t):
                    ch_rest -= 1
        if ch_rest > 0:
            nao_alocados.append((ped, ch_rest))
    return nao_alocados

def constroi_solucao(demanda):
    horario_temp = cria_horario_vazio(demanda)
    nao_aloc = alocar_demanda(horario_temp, demanda)
    return horario_temp, nao_aloc

melhor_horario = None
melhor_nao_alocados = None
menor_sobra = float("inf")
for i in range(NUM_ITERACOES):
    horario_cand, nao_aloc_cand = constroi_solucao(demanda)
    sobra = sum(x[1] for x in nao_aloc_cand)
    if sobra < menor_sobra:
        menor_sobra = sobra
        melhor_horario = horario_cand
        melhor_nao_alocados = nao_aloc_cand
print(f"Melhor solução após {NUM_ITERACOES} iterações: {menor_sobra} tempos não alocados (demanda).")


Melhor solução após 10000 iterações: 25 tempos não alocados (demanda).


In [16]:
# =========================================================================
# 7) MÉTODO DE AJUSTE FINO (FORÇADO) PARA ALINHAR OS TEMPOS REMANECENTES
# =========================================================================
def ajuste_fino(horario, nao_alocados):
    """
    Para cada pedido que ainda não teve todos os tempos alocados, tenta forçar
    a alocação em dias distintos (preferencialmente, sem repetir dia) – isto é,
    para cada turma, se o professor não tiver aula naquele dia, aloca no próximo slot
    disponível (que, por definição, é len(alocações)+1, garantindo consecutividade).
    Se ainda sobrar, permite a repetição do mesmo dia.
    """
    for ped, ch_rest in nao_alocados:
        # Primeiro, tentar alocar em dias ainda não usados pela demanda
        for d in dias_semana:
            if ch_rest <= 0:
                break
            # Se o professor já não foi alocado na turma nesse dia, tente forçar
            if not professor_turma_ja_usou_dia(horario, ped, d):
                alocs = horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)]
                proximo_slot = len(alocs) + 1
                if proximo_slot <= max(tempos):
                    if professor_disponivel(ped.prof, ped.turno, d, proximo_slot) and not professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, d, proximo_slot):
                        alocar_1tempo(horario, ped, d, proximo_slot)
                        ch_rest -= 1
        # Se ainda sobrar, permitir repetição (forçando a alocação mesmo que já haja aula nesse dia)
        if ch_rest > 0:
            for d in dias_semana:
                if ch_rest <= 0:
                    break
                alocs = horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)]
                proximo_slot = len(alocs) + 1
                if proximo_slot <= max(tempos):
                    if professor_disponivel(ped.prof, ped.turno, d, proximo_slot) and not professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, d, proximo_slot):
                        alocar_1tempo(horario, ped, d, proximo_slot)
                        ch_rest -= 1
        # Se ainda sobrar, a demanda permanece não alocada.
    return horario

In [17]:
# Aplica o ajuste fino se houver sobras
if menor_sobra > 0:
    print("Aplicando ajuste fino para encaixar os tempos remanescentes...")
    melhor_horario = ajuste_fino(melhor_horario, melhor_nao_alocados)
    # Recalcula sobras:
    novos_nao = alocar_demanda(melhor_horario, demanda)
    nova_sobra = sum(x[1] for x in novos_nao)
    print(f"Após ajuste fino, sobram {nova_sobra} tempos não alocados.")
else:
    print("Nenhum tempo remanescente a ajustar.")

Aplicando ajuste fino para encaixar os tempos remanescentes...
Após ajuste fino, sobram 94 tempos não alocados.


In [14]:
# =========================================================================
# 8) VERIFICAÇÃO: CALCULA TOTAL DE SLOTS VAZIOS NO QUADRO FINAL
# =========================================================================
def conta_alocacoes(horario):
    used = 0
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (sa, tu), dic in horario[tnr][niv][d].items():
                    used += len(dic)
    return used

# Supondo que cada turma tem 25 slots (5 dias x 5 tempos)
turmas_set = set((p.serie, p.turma) for p in demanda)
total_slots = len(turmas_set) * 25
usados = conta_alocacoes(melhor_horario)
tempos_vazios = total_slots - usados
print("----------------------------------------------")
print(f"Total de turmas: {len(turmas_set)} => {total_slots} slots disponíveis.")
print(f"Slots usados: {usados}")
print(f"Tempos vazios (slots não preenchidos): {tempos_vazios}")
print("----------------------------------------------")

----------------------------------------------
Total de turmas: 20 => 500 slots disponíveis.
Slots usados: 403
Tempos vazios (slots não preenchidos): 97
----------------------------------------------


In [ ]:
# =========================================================================
# 9) EXPORTAÇÃO: GERA PLANILHAS FINAL
#    (A) QUADRO DE HORÁRIOS POR TURMA (abas por (TURNO, NIVEL))
#    (B) QUADRO DE HORÁRIOS POR PROFESSOR (aba por professor)
#    (C) LISTA DE TEMPOS NÃO ALOCADOS (detalhado)
# =========================================================================

def gera_df_por_turma(horario):
    colunas_por_tn = defaultdict(set)
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (sa, tu), dic in horario[tnr][niv][d].items():
                    colunas_por_tn[(tnr,niv)].add(f"{sa}-{tu}")
    dfs_por_aba = {}
    for tnr in horario:
        for niv in horario[tnr]:
            turmas_cols = sorted(list(colunas_por_tn[(tnr,niv)]))
            linhas = []
            for d in dias_semana:
                for tm in tempos:
                    row_dict = {
                        "TURNO": tnr,
                        "NIVEL": niv,
                        "DIA": d,
                        "TEMPO": tm
                    }
                    for col in turmas_cols:
                        row_dict[col] = ""
                    for (sa, tu), dic in horario[tnr][niv][d].items():
                        if tm in dic:
                            (p, ds) = dic[tm]
                            col_name = f"{sa}-{tu}"
                            row_dict[col_name] = f"{p}+{ds}"
                    linhas.append(row_dict)
            df_aba = pd.DataFrame(linhas)
            dfs_por_aba[f"{tnr}_{niv}"] = df_aba
    return dfs_por_aba

def gera_df_por_professor(horario):
    prof_data = defaultdict(lambda: defaultdict(list))
    prof_cols = defaultdict(set)
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (sa, tu), dic in horario[tnr][niv][d].items():
                    for tm, (p, ds) in dic.items():
                        col = f"{sa}-{tu}({ds})"
                        prof_cols[p].add(col)
                        prof_data[p][(d,tm)].append( (tnr, niv, sa, tu, ds) )
    dfs_por_prof = {}
    for p in prof_data:
        cols = sorted(list(prof_cols[p]))
        linhas = []
        for d in dias_semana:
            for tm in tempos:
                row_dict = {
                    "PROFESSOR": p,
                    "DIA": d,
                    "TEMPO": tm
                }
                for c in cols:
                    row_dict[c] = ""
                if (d,tm) in prof_data[p]:
                    for (tnr, niv, sa, tu, ds) in prof_data[p][(d,tm)]:
                        cname = f"{sa}-{tu}({ds})"
                        row_dict[cname] = f"{tnr}-{niv}"
                linhas.append(row_dict)
        df_p = pd.DataFrame(linhas)
        dfs_por_prof[p] = df_p
    return dfs_por_prof

# Gerar planilhas
dfs_turma = gera_df_por_turma(melhor_horario)
dfs_prof = gera_df_por_professor(melhor_horario)

with pd.ExcelWriter("QUADRO_HORARIOS_TURMAS.xlsx") as writer:
    for aba, df_ in dfs_turma.items():
        df_.to_excel(writer, sheet_name=aba[:31], index=False)

with pd.ExcelWriter("QUADRO_HORARIOS_PROFESSORES.xlsx") as writer:
    for p, df_ in dfs_prof.items():
        sheetname = p[:31]
        df_.to_excel(writer, sheet_name=sheetname, index=False)

with pd.ExcelWriter("NAO_ALOCADOS_DETALHADO.xlsx") as writer:
    df_vaz = pd.DataFrame(vazios_detalhe) if 'vazios_detalhe' in globals() else pd.DataFrame()
    df_na = pd.DataFrame([{
        "SERVIDOR": ped.prof,
        "DISCIPLINA": ped.disc,
        "TURNO": ped.turno,
        "NIVEL": ped.nivel,
        "SERIE": ped.serie,
        "TURMA": ped.turma,
        "NAO_ALOCADO": c
    } for (ped, c) in melhor_nao_alocados if c > 0])
    # Juntar ambos (se houver vazios detalhados)
    df_comb = pd.concat([df_na, df_vaz], axis=0) if not df_vaz.empty else df_na
    df_comb.to_excel(writer, sheet_name="NAO_ALOCADOS", index=False)

print("Arquivos gerados:")
print(" - QUADRO_HORARIOS_TURMAS.xlsx (uma aba por Turno_Nivel)")
print(" - QUADRO_HORARIOS_PROFESSORES.xlsx (uma aba por professor)")
print(" - NAO_ALOCADOS_DETALHADO.xlsx (lista detalhada dos slots vazios e demandas não alocadas)")